In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#ARTIFICAL BEE
import numpy as np
from abc import ABCMeta
from six import add_metaclass
from copy import deepcopy

@add_metaclass(ABCMeta)
class ArtificialBee:
    TRIAL_DEFAULT_VALUE=0
    DEFAULT_PROBABILITY=0.0
    
    def __init__(self,objective_function):
        #intialising the position of food source where bee is present
        self.position=objective_fuction.custom_sample() 
        #specifying the objective funtion
        self.objective_function=objective_function
        #setting the boundaries of each food source attributes(variables)
        self.minf=objective_function.minf
        self.maxf=objective_function.maxf
        #setting the fitness value
        self.fitness=objective_function.evaluate(self.position)
        #intialising trial value
        self.trial=ArtificialBee.TRIAL_DEFAULT_VALUE
        #intialising probability 
        self.probability=ArtificialBee.DEFAULT_PROBABILITY
     
    #checking for boundary violation
    def evaluate_boundaries(self,position):
        if(position<self.minf).any() or (pos>self.maxf).any():
            position[position>self.maxf]=self.maxf
            position[position<self.minf]=self.minf
        return pos
    
    #function to update food source based on fitness value
    def update_bee(self,position,fitness):
        if fitness<=self.fitness:
            self.position=position
            self.fitness=fitness
            self.trial=0
        else:
            self.trial+=1;
    
#function to decide if food source should be reset
    def reset_bee(self,max_trials):
        if self.trial>=max_trials:
            self._reset_bee()

#function to reset the food source when the trial counter exceeds limit
    def __reset_bee(self):
        self.position=self.objective_function.custom_sample()
        self.fitness=self.objective_function.evaluate(self.position)
        self.trial = ArtificialBee.TRIAL_INITIAL_DEFAULT_VALUE
        self.probability = ArtificialBee.INITIAL_DEFAULT_PROBABILITY

In [ ]:
#EMPLOYEE BEE PHASE
class EmployeeBee(ArtificialBee):
    
    def explore(self,max_trials):
        if self.trial<=max_trial:
            #selecting a partner food source/solution randomly
            partner=np.random.choice(self.position)
            #generating a random value for phi between -1 and 1
            phi=np.random.uniform(low=-1,high=1,size=len(self.position))
            #generating new solution
            new_position=self.position+(self.position-component)*phi
            #checking for boundary violations
            new_position=self.evaluate_boundaries(new_position)
            #fitness value of new solution
            new_fitness=self.objective_function.evaluate(new_position)
            #updating the food source fitness value qualifies
            self.update_bee(new_position,new_fitness)
    
    #computation of fitness value
    def get_fitness(self):
        if self.fitness>=0:
            return 1/(1+self.fitness)
        else:
            return 1+np.abs(self.fitness)
    
    #probability computation
    def compute_probability(self,max_fitness):
        self.probability=self.get_fitness()/max_fitness

In [ ]:
#ONLOOOKER BEE PHASE
class OnLookerBee(ArtificialBee):
    
    def onloook(self,best_food_sources,max_trials):
        candidate=np.random,choice(best_food_sources)
        self.__exploit(candidate.position,candidate.fitnesss,max_trials)
    
    def __exploit(self,candidate,fitness,max_trials):
        if self.trial<=max_trial:
            #selecting a partner food source/solution randomly
            partner=np.random.choice(candidate)
            #generating a random value for phi between -1 and 1
            phi=np.random.uniform(low=-1,high=1,size=len(self.position))
            #generating new solution
            new_position=candidate+(candidate-component)*phi
            #checking for boundary violations
            new_position=self.evaluate_boundaries(new_position)
            #fitness value of new solution
            new_fitness=self.objective_function.evaluate(new_position)
        
        if new_fitness<=fitness:
            self.position=new_position
            self.fitness=new_fitness
            self.trial=0
        else:
            self.trial+=1

In [ ]:
#Artificial Bee Colony Algorithm 
class ABC:
    def __init__(self,objective_function,colony_size=30,no_of_iteration=5000,max_trials=100):
        self.colony_size=colony_size
        self.objective_function=objective_function
        self.no_of_iteration = no_of_itertion
        self.max_trials = max_trials
        self.optimal_solution = None
        self.optimality_tracking = []
        
    def __reset_algorithm(self):
        self.optimal_solution=None
        self.optimality_tracking = []
        
    def __update_optimality_tracking(self):
        self.optimality_tracking.append(self.optimal_solution.fitness)
        
    def __update_optimal_solution(self):
        new_optimal_solution=min(self.onlooker_bees+self.employee_bees,key=lambda bee: bee.fitness)
        
        if not self.optimal_solution:
            self.optimal_solution=deepcopy(new_optimal_solution)
        else:
            if new_optimal_solution.fitness<self.optimal_solution.fitness:
                self.optimal_solution=deepcopy(new_optimal_solution)
    
    def __initialize_employees(self):
        self.employee_bees=[]
        for i in range(self.colony_size//2):
            self.employee_bees.append(EmployeeBee(self.objective_function))
    
    def __initialize_onlookers(self):
        self.employee_bees=[]
        for i in range(self.colony_size//2):
            self.employee_bees.append(EmployeeBee(self.objective_function))
            
    def __calculate_probabilities(self):
        sum_fitness = sum(map(lambda bee: bee.get_fitness(), self.employee_bees))
        map(lambda bee: bee.compute_probability(sum_fitness), self.employee_bees)

    def __select_best_food_sources(self):
        self.best_food_sources =\
            filter(lambda bee: bee.prob > np.random.uniform(low=0, high=1),
                   self.employee_bees)
        while not self.best_food_sources:
            self.best_food_sources = \
                filter(lambda bee: bee.prob > np.random.uniform(low=0, high=1),
                       self.employee_bees)

    def __onlooker_bees_phase(self):
        map(lambda bee: bee.onlook(self.best_food_sources, self.max_trials),
            self.onlooker_bees)

    def __scout_bees_phase(self):
        map(lambda bee: bee.reset_bee(self.max_trials),
            self.onlooker_bees + self.employee_bees)

    def optimize(self):
        self.__reset_algorithm()
        self.__initialize_employees()
        self.__initialize_onlookers()
        for itr in range(self.no_of_iteration):
            self.__employee_bees_phase()
            self.__update_optimal_solution()

            self.__calculate_probabilities()
            self.__select_best_food_sources()

            self.__onlooker_bees_phase()
            self.__scout_bees_phase()

            self.__update_optimal_solution()
            self.__update_optimality_tracking()
            print("iter: {} = cost: {}"
                  .format(itr, "%04.03e" % self.optimal_solution.fitness))
    
        
    

        

        